In [1]:
from geopy.geocoders import Nominatim
from api_calls import get_strava_data
from get_data import prepare_data

In [3]:
import pandas as pd
#pd.options.display.max_rows = 999

import time
import datetime as dt
import numpy as np
from pathlib import Path
import requests
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import streamlit as st
import time

import plotly.express as px
import plotly.graph_objs as go
#import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None


In [13]:
# run this to update the activity data
#activities_raw = get_strava_data('strava_activities.csv')

# run this to use what's saved in directory
activities_raw = pd.read_csv('strava_activities.csv')

In [14]:
activities_clean = prepare_data(activities_raw)

In [15]:
def get_float_coords(elem):
    '''
    converts the latitude / longitude from a string that looks like a list to a functioning list of two floats
    '''
    try:
        return [float(coord) for coord in elem.split('[')[1].split(']')[0].split(',')]
    except ValueError:
        return np.nan

In [16]:
activities_clean['start_latlng'] = activities_clean['start_latlng'].apply(get_float_coords)


In [18]:
d = activities_clean.start_latlng[2]

In [21]:
f"{d[0]}, {d[1]}"

'45.52653598599136, -122.60245366953313'

In [22]:
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")


In [42]:
def city_state_county(row):
    try:
        lat = row['start_latlng'][0]
        long = row['start_latlng'][1]
        coord = f"{str(lat)}, {str(long)}"
        location = geolocator.reverse(coord, exactly_one=True)
        address = location.raw['address']
        city = address.get('city', 'N/A')
        state = address.get('state', 'N/A')
        country = address.get('county', 'N/A')
        row['city'] = city
        row['state'] = state
        row['county'] = county
    except TypeError:
        return np.nan
    return row

In [39]:
activities_clean = activities_clean.apply(city_state_country, axis=1)

2022-09-08 20:04:32.499 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /reverse?lat=45.527348&lon=-122.602561&format=json&addressdetails=1
2022-09-08 20:04:34.583 WARNING urllib3.connectionpool: Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002C21F5ED4B0>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)')': /reverse?lat=45.527348&lon=-122.602561&format=json&addressdetails=1
2022-09-08 20:05:38.946 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Rea

In [40]:
activities_clean.head()

,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,id,achievement_count,kudos_count,comment_count,...,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,suffer_score,SPM,city,state,country
start_time,,,,,,,,,,,,,,,,,,,,,
2022-09-08 19:30:32+00:00,Lunch Run,6.042206,3315.0,3357.0,226.37796,Run,7.776912e+09,0.0,1.0,0.0,...,147.2,161.0,290.026256,162.729664,0.0,64.0,1.158371,Portland,Oregon,United States
2022-09-07 18:57:18+00:00,Lunch Run,6.041088,3451.0,3473.0,649.60632,Run,7.771610e+09,3.0,2.0,0.0,...,150.5,167.0,408.792664,-32.152232,1.0,80.0,1.390901,Portland,Oregon,United States
2022-09-07 02:34:54+00:00,Evening Ride,9.460562,2605.0,2918.0,652.88716,Ride,7.767285e+09,10.0,2.0,0.0,...,NaN,NaN,614.829416,-184.383208,3.0,NaN,NaN,Portland,Oregon,United States
2022-09-06 17:03:45+00:00,Morning Run,3.022034,1582.0,1588.0,183.72704,Run,7.765027e+09,0.0,2.0,0.0,...,146.9,164.0,13.779528,-138.451448,0.0,29.0,1.099874,Portland,Oregon,United States
2022-09-05 16:21:30+00:00,Morning Hike,8.048160,10088.0,11687.0,1397.63784,Hike,7.760734e+09,0.0,4.0,0.0,...,99.8,143.0,1801.181160,436.351720,0.0,25.0,0.148692,N/A,California,United States


In [30]:
# this is a geolocation unsuccessful [43.775811, -121.515433]

location = geolocator.reverse('43.775811' + ',' + '-121.515433')


In [41]:
location.raw['address'].get('county', 'na')

'Deschutes County'

In [52]:
def add_city(elem):
    try:
        lat = str(elem[0])
        long = str(elem[1])
        location = geolocator.reverse(lat + ',' + long)
        return location.raw['address']['city']
    except KeyError:
        return 'geolocation unsuccessful'
    except TypeError:
        return np.nan

In [55]:
strava_raw.to_csv('activities_with_cities.csv', index=False)

In [53]:
strava_raw['city'] = strava_raw['start_latlng'].apply(add_city)

In [46]:
location = geolocator.reverse(lat + ',' + long)

In [45]:
lat = str(strava_raw.start_latlng[20][0])
long = str(strava_raw.start_latlng[20][1])

In [23]:
# start_latlng    end_latlng

Latitude = '45.5186696536839'
Longitude = '-122.6742431614548'
 
location = geolocator.reverse(Latitude+","+Longitude)
 
# Display
print(location)

United States Postal Service, Southwest Alder Street, Downtown, Portland, Multnomah County, Oregon, 97204, United States


In [40]:
location.raw['address']['city']

'Portland'

In [13]:
#strava_activities_clean.columns


In [73]:
strava_raw.city.value_counts(dropna=False)

Portland                    733
geolocation unsuccessful     79
NaN                          11
Beaverton                     2
Tucson                        2
Lake Oswego                   2
Austin                        1
Vancouver                     1
Name: city, dtype: int64

In [97]:
strava_raw_2.iloc[202]

resource_state                                                2
athlete                   {'id': 23871091, 'resource_state': 1}
name                                              Afternoon Run
distance                                                 5401.9
moving_time                                                1595
                                          ...                  
athlete.id                                                  NaN
athlete.resource_state                                      NaN
map.id                                                      NaN
map.summary_polyline                                        NaN
map.resource_state                                          NaN
Name: 202, Length: 62, dtype: object

In [96]:
strava_raw_2.iloc[nan_latlng_activities_list]

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,average_watts,kilojoules,device_watts,Unnamed: 0.1,Unnamed: 0,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state
182,2,"{'id': 23871091, 'resource_state': 1}",Lunch Activity,0.0,3676,3676,0.0,Elliptical,Elliptical,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,2,"{'id': 23871091, 'resource_state': 1}",Afternoon Run,5401.9,1595,1595,0.0,Run,Run,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323,2,"{'id': 23871091, 'resource_state': 1}",Treadmill 🥴🥴,9817.0,3480,3480,0.0,Run,Run,NaN,...,NaN,NaN,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN
452,2,"{'id': 23871091, 'resource_state': 1}",Afternoon Activity,0.0,3703,3703,0.0,Rowing,Rowing,NaN,...,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN
456,2,"{'id': 23871091, 'resource_state': 1}",Morning Activity,0.0,3675,3675,0.0,Rowing,Rowing,NaN,...,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN
459,2,"{'id': 23871091, 'resource_state': 1}",Afternoon Activity,0.0,3951,3951,0.0,Rowing,Rowing,NaN,...,NaN,NaN,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN
637,2,"{'id': 23871091, 'resource_state': 1}",Evening Run,71.5,20,20,0.0,Run,Run,0.0,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
638,2,"{'id': 23871091, 'resource_state': 1}",Evening Run,75.6,19,19,0.0,Run,Run,0.0,...,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN
646,2,"{'id': 23871091, 'resource_state': 1}",Afternoon ride,6115.5,1120,1120,0.0,Ride,Ride,10.0,...,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN
825,2,"{'id': 23871091, 'resource_state': 1}",Training run #1,4828.0,1895,1895,0.0,Run,Run,3.0,...,NaN,NaN,NaN,193.0,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
nan_latlng_activities_list = list(strava_raw[strava_raw.city.isna()][['start_latlng', 'city']].index)

In [28]:
test_df = pd.read_csv('activities_with_cities.csv')

In [29]:
# ah right because there is no latlng because these are all stationary activities (ellipticals, row machines, treadmills, etc.)
# so NaN it is tbh

test_df[test_df.city == 'geolocation unsuccessful'][['start_latlng', 'city']]
#testlong = str(test_df[test_df.city == 'geolocation unsuccessful'][['start_latlng', 'city']].start_latlng[202][1])

,start_latlng,city
13,"[30.16521248035133, -97.4509166367352]",geolocation unsuccessful
26,"[45.098740914836526, -121.75108842551708]",geolocation unsuccessful
40,"[45.696571692824364, -122.88972395472229]",geolocation unsuccessful
47,"[42.404475370422006, -124.42292651161551]",geolocation unsuccessful
48,"[42.40493897348642, -124.42184641957283]",geolocation unsuccessful
...,...,...
790,"[43.775811, -121.515433]",geolocation unsuccessful
791,"[43.894565, -121.369004]",geolocation unsuccessful
792,"[44.03065, -121.736076]",geolocation unsuccessful
800,"[45.686728, -121.265726]",geolocation unsuccessful


In [71]:
location.raw

{'place_id': 299497553,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 11175258,
 'lat': '30.16246135',
 'lon': '-97.45620405939258',
 'display_name': 'Wolfdancer Golf Club, 575, Hyatt Lost Pines, Bastrop County, Texas, 78612, United States',
 'address': {'leisure': 'Wolfdancer Golf Club',
  'house_number': '575',
  'road': 'Hyatt Lost Pines',
  'county': 'Bastrop County',
  'state': 'Texas',
  'ISO3166-2-lvl4': 'US-TX',
  'postcode': '78612',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['30.1563959', '30.1663972', '-97.4628052', '-97.4447212']}

In [7]:
activities = prepare_data()

In [2]:
activities = pd.read_csv('activities_clean.csv')

In [ ]:
activities.to_csv('activities_clean.csv')

In [8]:
activities.resample('W-Mon', closed='left').elapsed_time.sum()

start_time
2018-05-07 00:00:00+00:00    15731
2018-05-14 00:00:00+00:00     4962
2018-05-21 00:00:00+00:00        0
2018-05-28 00:00:00+00:00        0
2018-06-04 00:00:00+00:00        0
                             ...  
2022-07-11 00:00:00+00:00    26665
2022-07-18 00:00:00+00:00    26744
2022-07-25 00:00:00+00:00    15102
2022-08-01 00:00:00+00:00    32283
2022-08-08 00:00:00+00:00    26652
Freq: W-MON, Name: elapsed_time, Length: 223, dtype: int64

In [9]:
runs = activities[activities['sport_type'] == 'Run']
runs_2022 = runs[runs.index.year == 2022]

In [10]:
runs_2022.head(3)

,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,id,achievement_count,kudos_count,comment_count,...,max_speed,average_cadence,average_temp,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,suffer_score,SPM
start_time,,,,,,,,,,,,,,,,,,,,,
2022-08-07 17:14:37+00:00,Morning Run,11.412614,7926,7971,333.0,Run,7600095117,17,5,0,...,5.378,85.6,27.0,152.9,169.0,319.8,125.6,2,207.0,1.566995
2022-08-03 02:23:45+00:00,Evening Run,3.110341,1374,1374,31.0,Run,7573461254,3,4,1,...,4.463,87.9,29.0,163.0,177.0,83.6,63.2,3,51.0,2.227074
2022-08-01 18:56:46+00:00,Lunch Run,6.258065,3544,3544,74.0,Run,7566246906,0,2,0,...,4.440,83.5,30.0,147.5,159.0,83.8,48.0,0,72.0,1.218962


Miles Ran YTD

In [11]:
miles_run = runs_2022.distance.sum()

Elevation gain YTD

In [12]:
elev_gain = runs_2022.total_elevation_gain.sum()

Miles per day: 

In [13]:
mpd = runs_2022.distance.sum() / runs_2022.index[0].day_of_year

In [14]:
col1, col2, col3 = st.columns(3)
col1.metric("Miles Ran", miles_run, "-$1.25")
col2.metric("Elevation Gain", elev_gain, "0.46%")
col3.metric("Miles Per Day", mpd, "+4.87%")

2022-08-08 20:34:38.567 
  command:

    streamlit run c:\ProgramData\Miniconda3\envs\fitenv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [17]:
# my 6/12 run is showing up as 6/13

runs = activities[activities['sport_type'] == 'Run'].resample('W-Mon', closed='left').distance.sum().tail(12)
#st = strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].resample('M', closed='left').distance.sum().tail(15)

#st.index = st.index.strftime('%B %Y')
st.index = st.index.strftime('Week of %A, %b %d')

In [27]:
choices = {'W-Mon': '%A, %b %d %Y', 'M': '%B %Y', 'Y': '%Y'}


In [26]:
#list(choices.keys())#['W-Mon']
for choice in choices:
    print(choice, choices[choice])

choices[choice]

W-Mon Week of %A, %b %d %Y
M Month of %B %Y
Y Total for %Y


'Total for %Y'

In [31]:
hunger = ['w', 'm', 'y']

In [33]:
hunger.index('w')

0

In [30]:
choices = {'W-Mon': '%A, %b %d %Y', 'M': '%B %Y', 'Y': '%Y'}

for choice in choices: 
    df_slice = activities[activities['sport_type'] == 'Run'].resample(choice, closed='left').distance.sum().tail(5)
    df_slice.index = df_slice.index.strftime(choices[choice])
    print(df_slice)
    print()

start_time
Monday, Jul 11 2022    41.725922
Monday, Jul 18 2022    16.123271
Monday, Jul 25 2022    17.956401
Monday, Aug 01 2022    15.639016
Monday, Aug 08 2022    20.781020
Name: distance, dtype: float64

start_time
April 2022     132.981437
May 2022       137.867775
June 2022      141.490814
July 2022      107.788212
August 2022     20.781020
Name: distance, dtype: float64

start_time
2018      11.957790
2019      18.698496
2020     470.972734
2021    1491.678825
2022     909.215291
Name: distance, dtype: float64



### Strava Mobile Line Graph
Characteristics: 
* 12 weeks
* Select your activity at the top
* Header: Week of [Month day] - [Month day]
* Subheaders: 
  * Distance
  * Time in xh xxm
  * Elevation

In [61]:
run_activities = activities[activities['sport_type'] == 'Run']
run_by_week = run_activities.resample('W-Mon', closed='left')
run_by_week_agg = run_by_week.agg({'distance': 'sum', 'moving_time': 'sum', 'total_elevation_gain': 'sum'})
run_by_week_agg['total_elevation_gain'] = run_by_week_agg['total_elevation_gain'] * 3.28084
run_by_week_agg.moving_time = run_by_week_agg.moving_time.apply(to_time_readout)
#run_by_week_agg.moving_time = pd.to_datetime(run_by_week_agg.moving_time)


In [12]:
activities_by_week = activities.resample('W-Mon', closed='left')
activities_by_week_agg = activities_by_week.agg({'distance': 'sum', 'moving_time': 'sum', 'total_elevation_gain': 'sum'})
activities_by_week_agg['total_elevation_gain'] = activities_by_week_agg['total_elevation_gain'] * 3.28084
activities_by_week_agg.moving_time = activities_by_week_agg.moving_time.apply(to_time_readout)
#run_by_week_agg.moving_time = pd.to_datetime(run_by_week_agg.moving_time)

In [13]:
activities_by_week_agg

,distance,moving_time,total_elevation_gain
start_time,,,
2018-05-07 00:00:00+00:00,16.992982,1h 14m,564.304480
2018-05-14 00:00:00+00:00,17.412405,1h 19m,686.679812
2018-05-21 00:00:00+00:00,0.000000,0h 0m,0.000000
2018-05-28 00:00:00+00:00,0.000000,0h 0m,0.000000
2018-06-04 00:00:00+00:00,0.000000,0h 0m,0.000000
...,...,...,...
2022-07-11 00:00:00+00:00,41.725922,7h 19m,2936.351800
2022-07-18 00:00:00+00:00,35.875791,6h 12m,2116.141800
2022-07-25 00:00:00+00:00,35.211383,4h 5m,1604.330760


In [56]:
run_by_week_agg.moving_time = run_by_week_agg.moving_time.apply(to_time_readout)

In [52]:
to_time_readout(26363)

'7h 19m'

In [62]:
run_by_week_agg

,distance,moving_time,total_elevation_gain
start_time,,,
2018-08-06 00:00:00+00:00,11.957790,2h 3m,212.598432
2018-08-13 00:00:00+00:00,0.000000,0h 0m,0.000000
2018-08-20 00:00:00+00:00,0.000000,0h 0m,0.000000
2018-08-27 00:00:00+00:00,0.000000,0h 0m,0.000000
2018-09-03 00:00:00+00:00,0.000000,0h 0m,0.000000
...,...,...,...
2022-06-20 00:00:00+00:00,21.896833,3h 18m,1348.425240
2022-06-27 00:00:00+00:00,42.420821,7h 1m,2772.309800
2022-07-04 00:00:00+00:00,26.436405,4h 11m,2742.782240


In [10]:
def to_time_readout(element):
    time = element / 60 / 60
    hrs = int(time)
    mins = (time - hrs) * 60
    return f'{int(round(hrs, 0))}h {int(mins)}m'


In [59]:
int(7.99)

7

In [50]:
(7.32 - round(26363 / 60 / 60, 0)) * 60

19.200000000000017

In [73]:
run_by_week_agg.moving_time = run_by_week_agg.moving_time.apply(lambda x: y := x, f'{int(round(x, 0))}h {int(round((y - round(y)) * 60))}m')

SyntaxError: invalid syntax (2495979164.py, line 1)

In [31]:
run_by_week_agg.moving_time[4].minutes

AttributeError: 'Timedelta' object has no attribute 'minutes'

In [25]:
# week of 7/11 should be 7 hrs 19m in time
#run_by_week_agg.moving_time = dt.timedelta(seconds=run_by_week_agg.moving_time)


In [8]:
f'{int(round(time_wk, 0))}h {int(round((time_wk - round(time_wk)) * 60))}m'

'7h 19m'

In [21]:
print(dt.timedelta(seconds=26363))

7:19:23


In [58]:
# run_by_week_display = 
#run_by_week_agg.distance.round(2)
#run_by_week_agg.total_elevation_gain.round().astype(int)


In [5]:
time_wk = 26363/60/60

In [32]:
run_activities.iloc[[3]]

,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,id,achievement_count,kudos_count,comment_count,...,average_speed,max_speed,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,suffer_score,average_cadence,SPM
start_time,,,,,,,,,,,,,,,,,,,,,
2022-07-06 11:26:08+00:00,Lunch Run,4.038177,2164,2172,47.0,Run,7426810587,1,3,0,...,3.005,4.28,140.4,153.0,84.2,62.8,1,23.0,86.0,0.637708


start_time
2018-08-06 00:00:00+00:00     213
2018-08-13 00:00:00+00:00       0
2018-08-20 00:00:00+00:00       0
2018-08-27 00:00:00+00:00       0
2018-09-03 00:00:00+00:00       0
                             ... 
2022-06-20 00:00:00+00:00    1348
2022-06-27 00:00:00+00:00    2772
2022-07-04 00:00:00+00:00    2743
2022-07-11 00:00:00+00:00    2936
2022-07-18 00:00:00+00:00     197
Freq: W-MON, Name: total_elevation_gain, Length: 207, dtype: int32

In [33]:
# 154 ft
run_activities.total_elevation_gain[3] * 3.28084

154.19948

In [37]:
from datetime import timedelta

# create timedelta
sample_timedelta = timedelta(
    days=40,
    seconds=3,
    microseconds=2,
    milliseconds=1200,
    minutes=3,
    hours=23,
    weeks=3
)

In [41]:
sample_timedelta.seconds

82984

In [18]:
#st.index = st.index.strftime('%B %Y')
st

start_time
Week of Monday, May 02    41.244522
Week of Monday, May 09    26.253520
Week of Monday, May 16    20.777232
Week of Monday, May 23    43.479315
Week of Monday, May 30    30.362615
Week of Monday, Jun 06    33.258152
Week of Monday, Jun 13    33.822206
Week of Monday, Jun 20    21.896833
Week of Monday, Jun 27    42.420821
Week of Monday, Jul 04    26.436405
Week of Monday, Jul 11    41.725922
Week of Monday, Jul 18     6.080211
Name: distance, dtype: float64

In [9]:
#strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].head(30)
strava_activities_clean.columns

Index(['name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'sport_type', 'id', 'achievement_count',
       'kudos_count', 'comment_count', 'athlete_count', 'map', 'average_speed',
       'max_speed', 'average_heartrate', 'max_heartrate', 'elev_high',
       'elev_low', 'pr_count', 'suffer_score', 'average_cadence', 'SPM'],
      dtype='object')

In [16]:
strava_activities_raw.start_latlng[109]

'[45.52732522599399, -122.60262675583363]'

In [106]:
x = 'start_time'
y = 'average_speed'
df = strava_activities_clean[(strava_activities_clean.sport_type == 'Run') & (strava_activities_clean.distance.between(3.99, 8.5))]

fig = px.scatter(data_frame=df, 
                        y=y,   # choice 
                        x=x,
                        #hover_data=['distance', 'calories'], 
                        #labels=dict(x='Date', time='Time', avg_run_cadence='Average Run Cadence', avg_pace='Average Pace'), 
                        title=str(y + ' x ' + x),
                        #size='distance',
                        #color='distance',
                        template='plotly',
                        height=500, 
                        width=800, 
                        #trendline='ols'
                        )

In [107]:
fig.show()

In [76]:
type(strava_activities_clean['start_time'][2])

str

In [66]:
pd_dt_obj

Timestamp('2022-06-28 19:05:39+0000', tz='UTC')

In [65]:
pd_dt_obj + pd.Timedelta(3600, unit='sec')

Timestamp('2022-06-28 20:05:39+0000', tz='UTC')

In [67]:
regular_dt_obj

numpy.datetime64('2022-06-28T19:05:39.000000000')